<a href="https://colab.research.google.com/github/Narthanasiva/Narthanasiva/blob/main/paddleOCR_extracting_bill_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extracting bill information using paddle OCR.**




> **I used paddle OCR in this work. Because it extract most attributes correctly, thus it is more accurate than Tesseract and EasyOcr models for this task**



# 01. Install the paddle ocr. And to work with paddleocr model there is no need to import opencv and pillow libraries.

In [ ]:
!pip install paddleocr

In [ ]:
!pip install paddlepaddle

In [4]:
from paddleocr import PaddleOCR,draw_ocr

# 02. Next, convert the image to text using the ocr. Paddleocr supports Chinese, English, French, German, Korean and Japanese. I set the language to english("en") to peform this task.

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang='en')

In [ ]:
img_path = "bill.jpg"                                           #set the path of image
result = ocr.ocr(img_path, cls=True)

In [ ]:
for line in result:
    print(line)                                                 #Output is nested list

# 03. Arranged the text using the boundary box and save the string to the list as line by line

In [13]:
result_length = len(result)

result_line = []


text_1 = result[0]
text_1_height = text_1[0][3][1] - text_1[0][0][1]

i = 1
temp = [text_1]

while True:
  try:
    if not (text_1 in temp):
      temp.append(text_1)
  
    text_2 = result[i]

    if (text_1[0][0][1]  - (text_1_height / 2)) < text_2[0][0][1] < (text_1[0][0][1]  + (text_1_height / 2)):
      temp.append(text_2)
    else:
      temp = sorted(temp, key=lambda x: x[0][0][0])
      temp = [x[1][0] for x in temp]
      temp_string = " ".join(temp)
      result_line.append(temp_string)
      text_1 = result[i]
      temp = [text_1]
    i = i +1
  except:
    temp = sorted(temp, key=lambda x: x[0][0][0])
    temp = [x[1][0] for x in temp]
    temp_string = " ".join(temp)
    result_line.append(temp_string)
    break

In [14]:
print(result_line)

['MARCO POLO ORTIGAS MANILA', 'ApPaumo', 'FRONTIER ORTIGAS', 'HOTEL and RESORT CORP', '1600 Philippines', 'VAT REG TIN:008-611-772-000', 'LUNG HIN', 'Table:B1 OR333099-00', 'Date :08 Feb.15 Time 12:45', 'Staff ALEMIR Cover', '*1 JASMINE 178.57', '1 ST.CHX FEET X0 160.71', '*1 ST.PORK SIOMAI 196.43', '*1 ST.G RICE ABALONE 196.43', '1 ST.SPINACH DUMPLING 196.43', '*2 ST.SCALLOP DUMPLING 428.58', 'K1 BAKED BBQ PORK BUN 196.43', '*1 PANFRIED ONION CAKE 196.43', '1 FISH SCALLOP SP FX 339.29', '*1 MAR BEEF GARLIC 517.86', 'MPM DP 15% Bey -26.79', 'MPM Dining Privilege -809.44', 'Sub-TotaI 1770.93', 'VATable 1770.93', '10% S.C 260.72', '12% VAT 212.51', 'LTax Fd/B 49.59', 'Total: 2299.75']


# 04. list the consumed items and prices. I used python regular expression to findout the string that I need using r_pattern

In [ ]:
import re
items=[]

for i in result_line:
  x=re.search(r"[0-9]{1,5}[.,][0-9]{2}$",i)                
  if(x):
    rest_i = i.split(' ', 1)[1]
    items.append(rest_i)


#filtering the consumed items and prices
c_items=[]
for i in items:
  item_price={"item":"no data","price":"no data"}
  breaking_status=re.search(r'[0-9]*[%]',i)
  i=i.rsplit(' ',1)
  if (breaking_status):
    break
  item_price["item"]=i[0]
  item_price["price"]=i[1]

  c_items.append(item_price)

print(c_items)
    


# 05. Using regular expression I found the needed string from the result_line list. Then I foundout the needed attributes using grouping method of regular expression

In [42]:
for line in result_line:
  table_match = re.search(r'Table[ ]{0,1}[:]{0,1}[ ]{0,1}([A-Z][0-9])', line)
  if table_match:
    for value in table_match.groups():
            tble=value

  or_number_match = re.search(r'OR[#]{0,1}[ ]{0,1}[:]{0,1}[ ]{0,1}([0-9]{6}[-]{0,1}[0-9]{2})', line)
  if or_number_match:
    for value in or_number_match.groups():
            Order_number=value

  date_match = re.search(r'Date[ ]{0,1}[:]{0,1}[ ]{0,1}([0-3][0-9][ ]{0,1}[A-Z][a-z]{2}[.,]{0,1}[0-9]{2})', line)
  if date_match:
    for value in date_match.groups():
            dt=value
  
  time_match = re.search(r'Time[ ]{0,1}[:]{0,1}[ ]{0,1}([0-9]{2}[:]{0,1}[0-9]{2})', line)
  if time_match:
    for value in time_match.groups():
            tm=value

  staff_match = re.search(r'Staff[ ]{0,1}[:]{0,1}[ ]{0,1}([A-Z]{0,20})', line)
  if staff_match:
    for value in staff_match.groups():
            stff=value

  sub_total_match = re.search(r'Sub[ ]{0,1}[-]{0,1}[ ]{0,1}Tota[l,I]{0,1}[ ]{0,1}([0-9]{0,9}[.]{0,1}[0-9]{2})', line)
  if sub_total_match:
    for value in sub_total_match.groups():
            s_ttl=value
  
  total_match = re.search(r'Sub[ ]{0,1}[-]{0,1}[ ]{0,1}Tota[l,I]{0,1}[ ]{0,1}([0-9]{0,9}[.]{0,1}[0-9]{2})', line)
  if sub_total_match:
    for value in sub_total_match.groups():
            ttl=value
    

# 06. I created the output as a dictionary and I updated the keys and values

In [37]:
Output= {"table":"no data","staff":"no data","order number":"no data","date":"no data","time":"no data","consumeditems":"no data","sub total":"no data","total":"no data"}

# 07. Then, I updated the values by accesing the keys of output dictionary by the informations of bill

In [43]:
Output["table"]=tble
Output["staff"]=stff
Output["order number"]=Order_number
Output["date"]=dt
Output["time"]=tm
Output["consumeditems"]=c_items
Output["sub total"]=s_ttl
Output["total"]=ttl

In [44]:
print(Output)

{'table': 'B1', 'staff': 'ALEMIR', 'order number': '333099-00', 'date': '08 Feb.15', 'time': '12:45', 'consumeditems': [{'item': 'JASMINE', 'price': '178.57'}, {'item': 'ST.CHX FEET X0', 'price': '160.71'}, {'item': 'ST.PORK SIOMAI', 'price': '196.43'}, {'item': 'ST.G RICE ABALONE', 'price': '196.43'}, {'item': 'ST.SPINACH DUMPLING', 'price': '196.43'}, {'item': 'ST.SCALLOP DUMPLING', 'price': '428.58'}, {'item': 'BAKED BBQ PORK BUN', 'price': '196.43'}, {'item': 'PANFRIED ONION CAKE', 'price': '196.43'}, {'item': 'FISH SCALLOP SP FX', 'price': '339.29'}, {'item': 'MAR BEEF GARLIC', 'price': '517.86'}], 'sub total': '1770.93', 'total': '1770.93'}


# 08. Finally, I saved the output in a json file

In [45]:
import json

In [46]:
with open('output.json', 'w') as f:
    json.dump(Output, f)